In [2]:
import re  # Regular expressions
import nltk  # Natural Language Toolkit for text processing
import pandas as pd  # Data manipulation and analysis
from sklearn.model_selection import train_test_split  # Splitting data into train and test sets
from sklearn.neural_network import MLPClassifier  # Multi-layer Perceptron classifier
from sklearn.preprocessing import LabelEncoder  # Encoding labels
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score  # Model evaluation metrics
from sklearn.feature_extraction.text import TfidfVectorizer  # Text vectorization
import pyaudio  # Audio input
import numpy as np  # Numerical operations
import speech_recognition as sr  # Speech recognition library

# Load NLTK stopwords (common words to be removed from text)
nltk.download('stopwords')

# Load your dataset from a CSV file named 'training.csv'
dataset = pd.read_csv('training.csv')

# Initialize an empty list to store processed text
corpus = []

# Loop through each row in the dataset
for i in range(len(dataset)):
    text = str(dataset.iloc[i, 0])  # Extract the text from the first column
    text = re.sub('[^a-zA-Z]', ' ', text).lower()  # Remove non-alphabet characters and convert to lowercase
    words = text.split()  # Split the text into words
    text = ' '.join(words)  # Join the words back into a string
    corpus.append(text)  # Append the processed text to the corpus list

# Store the processed text in a variable
processed_text = corpus




# Use TF-IDF Vectorization with a limited number of features (e.g., 5000)
max_features = 5000
vectorizer = TfidfVectorizer(max_features=max_features)
X = vectorizer.fit_transform(processed_text).toarray()
print(X.shape)

# Extract emotion labels from the dataset
emotion_detection_y = dataset.iloc[:, 1].values

# Encode the emotion labels into numerical format
le = LabelEncoder()
emotion_detection_y = le.fit_transform(emotion_detection_y)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, emotion_detection_y, test_size=0.2, random_state=0)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adityaprasad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(16000, 5000)


In [3]:
# Create a Multi-layer Perceptron (MLP) classifier with 2 hidden layers (100 neurons, 50 neurons)
clf = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=300, random_state=0)

# Train the MLP classifier on the training data
clf.fit(X_train, y_train)

# Predict the labels for the test data
y_pred = clf.predict(X_test)

# Calculate and print the confusion matrix and accuracy score
conf_matrix = confusion_matrix(y_test, y_pred)
acc_score = accuracy_score(y_test, y_pred)
pre_score = precision_score(y_test, y_pred, average = "macro")
print("Confusion Matrix:\n", conf_matrix)
print("Accuracy Score:", acc_score)
print("Precision Score:", pre_score)

Confusion Matrix:
 [[799  36   7  44  17   2]
 [ 37 930  58  14  10   4]
 [  9  63 188   5   6   0]
 [ 45  16   2 382  14   0]
 [ 34  26  10  37 275  15]
 [ 10  14   2   0   8  81]]
Accuracy Score: 0.8296875
Precision Score: 0.8061175322488671


In [37]:
# Initialize PyAudio for audio input
p = pyaudio.PyAudio()

# Define audio capture parameters
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 5

# Start the audio stream
stream = p.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)
print("Listening...")

# Record audio for a specified duration
frames = []
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("Finished recording.")

# Stop and close the audio stream
stream.stop_stream()
stream.close()

# Process the live audio by converting it to a NumPy array
live_audio = np.frombuffer(b''.join(frames), dtype=np.int16)

# Use SpeechRecognition to transcribe the audio to text
recognizer = sr.Recognizer()
live_audio_data = sr.AudioData(live_audio.tobytes(), RATE, live_audio.dtype.itemsize)

# Recognize the audio and convert it to text using Google Speech Recognition
try:
    recognized_text = recognizer.recognize_google(live_audio_data)
    print("Audio to Text:", recognized_text)
except sr.UnknownValueError:
    print("Google Speech Recognition could not understand audio")
except sr.RequestError as e:
    print(f"Could not request results from Google Speech Recognition service; {e}")

# Convert the live text to a numerical format using TF-IDF Vectorizer
live_vector = vectorizer.transform([recognized_text]).toarray()
#print("Live Vector:", live_vector)

Listening...
Finished recording.
Audio to Text: I love you try something else I am disappointed


In [38]:

# Predict the emotion label for the live audio using the trained classifier
predicted_labels = clf.predict(live_vector)

# Define a mapping of predicted labels to emotions
emotion_mapping = {
    0: "sad",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

# Map the predicted label to the corresponding emotion
predicted_emotion = emotion_mapping.get(predicted_labels[0], "Unknown")

# Print the predicted emotion
print("Predicted Emotion:", predicted_emotion)

# Create a dictionary with recognized_text as keys and predicted_emotion as values
'''data_dict = {'text': [recognized_text], 'label': [predicted_emotion]}

# Create a DataFrame from the dictionary
new_data_df = pd.DataFrame(data_dict)

# Append the new data to the original dataset
updated_dataset = pd.concat([dataset, new_data_df], ignore_index=True)

# Save the updated DataFrame to the CSV file
updated_dataset.to_csv('training.csv', index=False)

print("Updated dataset saved to training.csv.")'''

Predicted Emotion: sad


'data_dict = {\'text\': [recognized_text], \'label\': [predicted_emotion]}\n\n# Create a DataFrame from the dictionary\nnew_data_df = pd.DataFrame(data_dict)\n\n# Append the new data to the original dataset\nupdated_dataset = pd.concat([dataset, new_data_df], ignore_index=True)\n\n# Save the updated DataFrame to the CSV file\nupdated_dataset.to_csv(\'training.csv\', index=False)\n\nprint("Updated dataset saved to training.csv.")'